In [ ]:
# call __init_ to set notebook seed
import set_notebook_env
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import statsmodels.formula.api as smf
from scipy.stats import mannwhitneyu
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from utils.modeling_helpers import ModelHelper
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.decomposition import PCA


from collections import defaultdict
import numpy as np

import set_notebook_env
loader, plot_utils = set_notebook_env.set_env(
    data_dir="../data/", 
    min_ratings=0, 
    max_ratings=None,
    drop_inconsistent_gender = True
)

SEED = np.random.get_state()[1][0]

loader.initial_setup()
loader.add_tag_intensity_score()
prepared_df = loader.add_gender_code(loader.prepared_df)
tag_columns = loader.tag_cols
prepared_df.columns

tag_intensity_score = loader.find_agg_tag_columns("tag_intensity_score")
loader.add_column_zscores_standardized(columns = tag_intensity_score)
z_score_cols = loader.find_agg_tag_columns("zscore_tag_intensity_")
z_score_cols

prepared_df = loader.prepared_df

MODEL_HELPER = ModelHelper(df=prepared_df, random_seed=SEED)
